In [43]:
import numpy as np
import pandas as pd
import re
import torch
from torch.utils.data import DataLoader, TensorDataset

In [2]:
ev_path = "/content/drive/My Drive/DevC/process materials/Englishwords.xlsx"
sf_path = "/content/drive/My Drive/DevC/process materials/Shortform.xlsx"

englishwords = pd.read_excel(ev_path, index_col = "English")
shortform = pd.read_excel(sf_path, index_col= "Short")

In [3]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)
def preprocess(text):
  #bỏ tag html và emoji
  text = re.sub('<[^>]*>', '', text)
  text = deEmojify(text)

  #thay chữ cái viết hoa thành viết thường
  text = text.lower()

  #xóa dấu ngắt câu, xóa link và các chữ có chứa chữ số
  clean_text = []
  punc_list = '.,;:?!\|/&@`~()-_@#$%^*'
  for w in (text.split()):
    if "http" in w:
      continue
    clean_text.append(w)
  text = ' '.join(clean_text)
  for punc in punc_list:
    text = text.replace(punc, ' ')

  #xóa bỏ các chữ cái lặp liên tiếp nhau (đỉnhhhhhhhhhh, vipppppppppppppppp)
  length = len(text)
  char = 0
  while char <length-1:
    if text[char] == text[char+1]:
      text = text[:char]+text[char+1:]
      #print(text)
      length-=1
      continue
    char+=1  
  numbers = ["không", "một", "hai", "ba", "bốn", "năm", "sáu", "bảy", "tám", "chín"]
  #chuyển đổi các từ tiếng anh và viết tắt thông dụng sang tiếng Việt chuẩn:
  text_split = text.split()
  for i, w in enumerate(text_split):
    if w in englishwords.index:
      text_split[i] = str(englishwords.loc[w, "Vietnamese"])
    if w in shortform.index:
      text_split[i] = str(shortform.loc[w, "Long"])
    if w.isdigit():
      text_split[i] = ' '.join([numbers[int(c)] for c in w]) 
  text = ' '.join(text_split)

  #loại bỏ tất cả các kí tự đặc biệt còn lại
  digits_and_characters = 'aăâbcdđeêfghijklmnoôơpqrstuưvxywzáàảãạắằẳẵặấầẩẫậéèẻẽẹếềểễệíìỉĩịóòỏõọốồổỗộớờởỡợúùủũụứừửữựýỳỷỹỵ0123456789 '
  text = ''.join([i for i in text if i in digits_and_characters])
  return text

In [4]:
# read data from text files
data = pd.read_csv("/content/drive/My Drive/DevC/datasets/tiki.csv")
#print(data.head(5))
data = data[["content", "label"]]
data = data[data["label"].notnull()]
print(data.head(10))

                                             content  label
0  Mua hàng chốt đơn nhận tiền rất nhanh, không k...    0.0
1  nhưng nhận hàng thì 5 ngày rồi chưa kích foajt...   -1.0
2           Rất không hài lòng với hậu mãi như Tiki.    0.0
3  Lúc mua cái này mình phân vân lắm, tại tới tận...    1.0
4  chất lượng tai nghe rất tốt, nhưng:\nđồ công n...   -1.0
5  Tiki giao hàng nhanh chóng, sản phẩm chính hãn...    0.0
6  Tuy nhiên với những món hàng giá trị thì tiki ...   -1.0
7            Để tránh giao hàng nhầm cho người khác.    0.0
8  Bước đầu nhận hàng hài lòng vì sp còn nguyên s...    1.0
9                      Sử dụng nghe tốt, chất lượng.    1.0


In [5]:
from gensim.models.phrases import Phrases, Phraser
bigram = Phraser.load("/content/drive/My Drive/DevC/saves/bigram.pkl")

In [6]:
for record in data.index:
  data.loc[record,"content"] = preprocess(data.loc[record, "content"])

In [7]:
print(data.head(10))

                                             content  label
0  mua hàng chốt đơn nhận tiền rất nhanh không kị...    0.0
1  nhưng nhận hàng thì năm ngày rồi chưa kích foa...   -1.0
2            rất không hài lòng với hậu mãi như tiki    0.0
3  lúc mua cái này mình phân vân lắm tại tới tận ...    1.0
4  chất lượng tai nghe rất tốt nhưng đồ công nghệ...   -1.0
5  tiki giao hàng nhanh chóng sản phẩm chính hãng...    0.0
6  tuy nhiên với những món hàng giá trị thì tiki ...   -1.0
7             để tránh giao hàng nhầm cho người khác    0.0
8  bước đầu nhận hàng hài lòng vì sp còn nguyên s...    1.0
9                        sử dụng nghe tốt chất lượng    1.0


In [9]:
data = data.sample(frac = 1).reset_index(drop = True)
print(data.head(10))

                                             content  label
0  sản phẩm có chất lượng tốt chất âm hay hơn so ...    1.0
1  âm thanh thì m chưa thử nhưng thấy đánh giá kh...    1.0
2                              đeo thoải mái nhẹ tay    1.0
3  đây là nhận định cá nhân of tôi khj mới xài đư...    0.0
4                          tai nghe rẻ xài cũng được    1.0
5                        hàng đẹp mọi người ạ mua đi    1.0
6                                     nghe khá là ổn    1.0
7                                        xai cung ổn    1.0
8  hàng nguyên seal cấu hình quá tốt với màn supe...    1.0
9               hôm nay nhận được thì lại bị như vậy    0.0


In [11]:
split = 0.8
num_record = data.shape[0]
data_train = data.loc[:num_record*split]
data_test = data.loc[num_record*split:]

In [15]:
comments_train = data_train["content"].to_list()
labels_train = data_train["label"].astype(int).to_list()
comments_test = data_test["content"].to_list()
labels_test = data_test["label"].astype(int).to_list()
print(comments_train[:5])
print(labels_train[:5])


['sản phẩm có chất lượng tốt chất âm hay hơn so với tai nghe bóc hộp của một số máy khuyên mọi người nên mua', 'âm thanh thì m chưa thử nhưng thấy đánh giá khá ổn', 'đeo thoải mái nhẹ tay', 'đây là nhận định cá nhân of tôi khj mới xài được mấy ngày', 'tai nghe rẻ xài cũng được']
[1, 1, 1, 0, 1]


In [16]:
def tokenize(comments):
  splitComments = []
  for c in comments:
    s = c.split()
    splitComments.append(s)

  nsc = bigram[splitComments]
  nc = [' '.join(c) for c in nsc]
  return nc

In [18]:
comments_train = tokenize(comments_train)
comments_test = tokenize(comments_test)
print(comments_train[:5])
print(comments_test[:5])

['sản_phẩm có chất lượng tốt chất âm hay hơn so với tai nghe bóc hộp của một số máy khuyên mọi người nên mua', 'âm thanh thì m chưa thử nhưng thấy đánh giá khá ổn', 'đeo thoải_mái nhẹ tay', 'đây là nhận định cá nhân of tôi khj mới xài được mấy ngày', 'tai nghe rẻ xài cũng được']
['', 'chỉ nên mua sản_phẩm cho mục_đích sử_dụng để nghe gọi hoặc làm máy phụ', 'tiki giao hàng đóng gói kỹ', 'rất bực', 'dây dẹt cao_su chống rối độ bền thì chưa biết như vậy cũng không nên kỳ vọng quá nhiều với giá này']


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
tfidf = TfidfVectorizer(ngram_range= (1,3))
tfidf_train = tfidf.fit_transform(comments_train)
tfidf_test = tfidf.transform(comments_test)
print(tfidf_train.shape)

(8341, 97229)


In [20]:
print(tfidf.vocabulary_)

Output hidden; open in https://colab.research.google.com to view.

In [27]:
print(tfidf_test.shape)

(2085, 97229)


In [78]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components = 100)
svd_train = svd.fit_transform(tfidf_train)
svd_test = svd.transform(tfidf_test)

In [79]:
print(svd_train.shape)

(8341, 100)


In [80]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter= 1000)
lr.fit(svd_train, labels_train)
lr.score(svd_test, labels_test)

0.6882494004796164

In [81]:
from sklearn.metrics import f1_score, classification_report
print(f1_score(labels_test, lr.predict(svd_test), average= "macro"))
print(classification_report(labels_test, lr.predict(svd_test)))

0.6425073139638506
              precision    recall  f1-score   support

          -1       0.70      0.60      0.65       400
           0       0.59      0.43      0.50       593
           1       0.72      0.86      0.78      1092

    accuracy                           0.69      2085
   macro avg       0.67      0.63      0.64      2085
weighted avg       0.68      0.69      0.68      2085



In [69]:
svd_train = svd_train.reshape((svd_train.shape[0], svd_train.shape[1], 1))
svd_test = svd_test.reshape((svd_test.shape[0], svd_test.shape[1], 1))
print(svd_train.shape)

(8341, 100, 1)


In [70]:
data_train = TensorDataset(torch.from_numpy(svd_train), torch.from_numpy(np.array(labels_train)))
data_test = TensorDataset(torch.from_numpy(svd_test), torch.from_numpy(np.array(labels_test)))

In [71]:
batch_size = 50
train_loader = DataLoader(data_train, shuffle=True, batch_size=batch_size, drop_last= True)
test_loader = DataLoader(data_test, shuffle= True, batch_size= batch_size, drop_last = True)

In [72]:
from torch import nn as nn

In [73]:
class SentimentLSTM(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, output_size, embedding_dim, hidden_dim, n_layers, n_cell, drop_prob = 0.2):
        """
        Initialize the model by setting up the layers.
        """
        super().__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_size = embedding_dim,hidden_size = hidden_dim, num_layers = n_layers, batch_first = True, dropout = drop_prob, bidirectional = False)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.softmax = nn.Softmax(dim = 1)        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)
        lstm_out, hidden = self.lstm(x, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        out = self.dropout(lstm_out)
        out = self.fc(out)
        out = out.contiguous().view(batch_size, -1, output_size)
        out = out[:, -1, :]
        out = self.softmax(out)
        return out, hidden
    
    
    def init_hidden(self, batch_size, train_on_gpu = False):
        ''' Initializes hidden state '''
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().float().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().float().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().float(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().float())
        
        return hidden

In [74]:
output_size = 3
embedding_dim = 1
hidden_dim = 128
n_layers = 2
drop_prob = 0.2
n_cell = 1000
net = SentimentLSTM(output_size, embedding_dim, hidden_dim, n_layers, n_cell, drop_prob)
print(net)

SentimentLSTM(
  (lstm): LSTM(1, 128, num_layers=2, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=128, out_features=3, bias=True)
  (softmax): Softmax(dim=1)
)


In [75]:
# loss and optimization functions
lr=0.003
train_on_gpu = False
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr)


# training params

epochs = 8
counter = 0
print_every = 60
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()
net.train()
# train for some number of epochs
for e in range(epochs):
    
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1
        #print(inputs.shape)
        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()
        # get the output from the model
        inputs = inputs.float()
        
        output_prob,h = net(inputs, h)

        # calculate the loss and perform backprop
        labels = labels+1
        #print(labels)
        loss = criterion(output_prob, labels)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in test_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                inputs = inputs.float()
                output, val_h = net(inputs, val_h)
                labels = labels+1
                val_loss = criterion(output, labels)

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/8... Step: 60... Loss: 1.046398... Val Loss: 1.018260
Epoch: 1/8... Step: 120... Loss: 1.032267... Val Loss: 1.019697
Epoch: 2/8... Step: 180... Loss: 0.969258... Val Loss: 1.017767
Epoch: 2/8... Step: 240... Loss: 0.973113... Val Loss: 1.015620
Epoch: 2/8... Step: 300... Loss: 0.960859... Val Loss: 1.016487
Epoch: 3/8... Step: 360... Loss: 1.024458... Val Loss: 1.015674
Epoch: 3/8... Step: 420... Loss: 0.946662... Val Loss: 1.014761
Epoch: 3/8... Step: 480... Loss: 1.066250... Val Loss: 1.018903
Epoch: 4/8... Step: 540... Loss: 1.014881... Val Loss: 1.015520
Epoch: 4/8... Step: 600... Loss: 0.895563... Val Loss: 1.017195
Epoch: 4/8... Step: 660... Loss: 0.991815... Val Loss: 1.018950
Epoch: 5/8... Step: 720... Loss: 1.014780... Val Loss: 1.016425


KeyboardInterrupt: ignored